### Hyperspy notebook to generate video from a series of TEM image
#### by Martial Duchamp NTU-MSE, Singapore m.duchamp@ntu.edu.sg

In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import time
import csv
from os import listdir
from os.path import isfile, join
import pandas as pd
import matplotlib.animation as animation
import matplotlib as mpl
from scipy import interpolate
from scipy import ndimage
#import numpy as np, cv2
import matplotlib.patches as patches

mpl.rcParams['axes.linewidth'] = 3.0 #set the value globally
#mpl.use("Agg")

In [2]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [3]:
#import sys 
#sys.path.insert(0, '/home/martial/Data/programs/Hyperspy_ungit')
import hyperspy.api as hs

In [4]:
hs.hyperspy.Release.info

'\n    H y p e r S p y\n    Version 1.5.2\n\n    http://www.hyperspy.org\n\n    '

### Read TEM images

In [21]:
path = '/mnt/nisbfs/Data/Vaso/2020-12-28/TEM_time-resolved/Test-3'
TEM = hs.load(path+'/image_1,*.dm4')

In [22]:
TEM

<Signal2D, title: image_1, temperature_394.377, dimensions: (|4096, 4096)>

In [7]:
TEM.plot()

In [24]:
TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical

├── Applied Voltage(V) = 1.04
├── Measured Current(A) = 0.00559759
├── Resistance(Ohms)  = 185.79424359411817
└── Temperature(K)  = 394.3769743764727

In [12]:
TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Applied_VoltageV

6.05816e-08

In [13]:
TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Measured_CurrentA

0.1

In [17]:
TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.ResistanceOhms

140.97560975609755

In [ ]:
R =b*T+c

In [25]:
b=0.41
c=39

In [26]:
R=TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.ResistanceOhms
T= (R-c)/b
T

358.03474047345895

In [27]:
Elec_data = np.zeros((211, 5))

In [ ]:
for i in range(0,np.shape(Elec_data)[0]):
    TEM = hs.load(path+'/image_'+str(i)+',*.dm4')
    Elec_data[i-1,0]=TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Measured_CurrentA
    Elec_data[i-1,1]=TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Applied_VoltageV
    Elec_data[i-1,2]=TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.ResistanceOhms
    Elec_data[i-1,3]=(TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.ResistanceOhms-c)/b   
    Elec_data[i-1,4]=i

In [19]:
np.savetxt(path +'/Elec_data.txt', Elec_data)
np.save(path +'/Elec_data.npy', Elec_data)

In [20]:
fig, ax1 = plt.subplots()
ax1.set_xlabel('Image nbr')
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax1.plot(Elec_data[:,3],Elec_data[:,1], marker='o', linestyle='dashed', linewidth=2,label='Voltage')
ax1.set_ylabel('Voltage [V]')

ax2.plot(Elec_data[:,3],Elec_data[:,0], 'v', label='Current', color='green')
ax2.set_ylabel('Current [A]')
fig.legend(loc = "upper center", bbox_to_anchor=(0.7, 0.95))
fig.tight_layout()
plt.savefig(path+'/voltage_img-nbr.png')
plt.show()

In [ ]:
Elec_data[29,0]/Elec_data[29,1]

R= TEM[1].original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Applied_VoltageV / \
    TEM[1].original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Measured_CurrentA

T = (R -c)/b
T

### Video

In [ ]:
FFMpegWriter = animation.writers['ffmpeg']
metadata = dict(title=path, artist='M. Duchamp')
writer = FFMpegWriter(fps=0.5, metadata=metadata, bitrate=200)

In [ ]:
vmin_mul = 0.
vmax_mul = 2.5

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1)
im = axes.imshow(ndimage.gaussian_filter(TEM[0].data,6), cmap='gray', \
                  vmin=vmin_mul*np.mean(TEM[0].data), vmax=vmax_mul*np.mean(TEM[0].data))

R= TEM[0].original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Applied_VoltageV / \
    TEM[0].original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Measured_CurrentA

if R <0.001: R=b*130+c

textbox = plt.text(0.8, 0.8, \
    '{0:.0f}'.format((R - c)/b) +'K', \
    fontsize=15, verticalalignment='top', backgroundcolor='white') 

plt.show()

In [ ]:
with writer.saving(fig, "Test-3.mp4", dpi=200):
    for i in range(0,TEM.count(0)):
        im = axes.imshow(ndimage.gaussian_filter(TEM[i].data,6), cmap='gray', \
                         vmin=vmin_mul*np.mean(TEM[i].data), vmax=vmax_mul*np.mean(TEM[i].data))           

        R= TEM[i].original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Applied_VoltageV / \
            TEM[i].original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Measured_CurrentA
        if R <0.001: R=b*130+c
            
        textbox = plt.text(0.8, 0.8, '{0:.0f}'.format((R - c)/b) +'K', \
            fontsize=15, verticalalignment='top', backgroundcolor='white')        
        
        writer.grab_frame()
        textbox.remove() 
        print(i)
        
    for j in range(0,3):
        im = axes.imshow(ndimage.gaussian_filter(TEM[i].data,6), cmap='gray', \
                         vmin=vmin_mul*np.mean(TEM[i].data), vmax=vmax_mul*np.mean(TEM[i].data))           

        R= TEM[i].original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Applied_VoltageV / \
            TEM[i].original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Measured_CurrentA
        if R <0.001: R=b*130+c
            
        textbox = plt.text(0.8, 0.8, '{0:.0f}'.format((R - c)/b) +'K', \
            fontsize=15, verticalalignment='top', backgroundcolor='white')  
        
        writer.grab_frame()
        textbox.remove() 
        print(i)